In [3]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.express as px
import geopandas
import plotly.graph_objects as go
from tqdm import notebook

# station 정보
station = pd.read_csv('D:\git_local_repository\data_for_project\seoul_bike\seoul_bike_station_01_12.csv',encoding='CP949',index_col=0)
station.rename(columns={'대여소 번호' : 'id'}, inplace=True)


#plotly continuous_color
continuous_color = pd.read_csv('data/plotly_continuous_color_codes.csv')

##생활권 분석
### 가설
1. 해당 대여소가 몰리는 곳인지 퍼지는 곳인지 분류할 수 있음
    - 몰리는 곳에 대한 정의는 ? 다양한 대여소에서 온다. 나가는 따릉이보다 들어가는 따릉이가 더 많다.
    
    - 지역구간 단방향 이동인지 양방향 이동인지 파악 할 수 있음. 어느정도 범위는 양방향이고 음수이면 왼쪽방향 양수이면 오른쪽 방향이라는 걸 알 수 있겠지? 
    - 몰리는 곳 설정 + 흐름을 합치면 뭔가 재밌는 그림이 나올거 같다는 생각이다.
    => 기상 흐름도를 채용하면 되지 않을까?

<br>

1. 어디가 생활반경인지 보다는 물리적으로 지역임에도 심리가 먼 지역이 어디인지를 보는데 초점을 맞춤. 
    - 뭐가 됐던 사람들이 가지는 근처 지역에 대한 심리가 실제로 얼추 맞는지 살펴보면 되지 않을까?
    
    - 심리가 먼 지역에 대한 정의는? 물리적으로 가까움에도 top 20에 포함되지 않은 지역 (top10,top20,top50 다 생각해보자)
    - 20분~ 30분 거리를 등고선으로 하면 활동 범위를 알 수 있으려나? 아무리 가까워도 안가는 곳이라면 20-30분 거리로 넓혀봤자 포함안된다.

<br>


1. 따릉이로 안가본 서울 도심 체험하기
    - 본인이 주로 애용하는 대여소를 기점으로 사람들이 자주 가는 순으로 여행지를 택할 수 있게 하자.
    - 많은 사람들이 이용한다는 말은 그만큼 따릉이로 이동하기 어렵지 않다는 말
  
    - 방법

        1.원하는 따릉이 대여소 검색

        2.table로 선택 되면 그롷게 하고 안되면 드롭다운 방식으로 보여주기

        3.선택하면 collector로 올라가게 함

        4.추천 여행지 우측 테리블에 정렬
        
        5.클릭하면 해당 지역으로 이동

<br>


1. 따릉이로 최적 거리 이동하기
    - 내가 A대여소에서 어느 지점으로 간다고 지역을 누르면 반경에 있는 따릉이 대여소 몇 군데를 골라 예상 운행 시간을 알려준다.
      
      사람들이 가장 많이 이용한 대여소 또는 최단 거리가 걸리는 대여소를 알려주면 될 듯
      
    - 많은 사람들이 이용한다는 말은 그만큼 따릉이로 이동하기 어렵지 않다는 말
    - 지도앱 예상 시간과 따릉이 이용 예상시간 차이는 매우 크다. 
  
    -  따릉이로 가기 좋은 길인지, 얼마정도 시간이 걸리는지 예상용도
        네이버 지도 예상시간은 따릉이를 이용하기엔 너무 빠르다.
    - 네이버 지도와 데이터를 바탕으로 최소 이동시간이 소요되는 지역을 추천할 수 있다.

<br>


### 필요한 사항 정리
1. 몰리는 곳 정의
예시) (여의나루 역 2번출구 => 다른 지역) 총 id 개수 와  (다른 지역 => 여의나루 역 2번 출구) 총 id 개수 비교 

    & (여의나루 2번 출구 => 다른 지역으로 이동한 따릉이 대여 개수) 와 (다른 지역에서 여의나루 2번 출구로 이동한 따릉이 개수) 비교

    * 그러려면 st_id2가 여의나루인 경우 id 개수, 따릉이 대여 대수가 필요하고 st_id1이 여의나루인 상태에서 전 지역으로 이동한 개수와 따릉이 대여 개수가 필요하겠네

    * 개수만 새는거니까 그렇게 오래 걸리지 않겠지?

    * 이때 10개,20개,30개 이상 데이터가 있는 경우 비교 하는 것도 좋고 아니면 전체 비율에서 0.005% 이상인 비율만 찾아도 될거 같다는 생각이네

<br>


2. 대여소간 이동 흐름 분석해보기 
예시)  월촌초 <=> 신목동역을 예로들면 월촌초 => 신목동 가는 횟수와 신목동 => 월촌초 가는 횟수를 비교해서 $\frac{월=>신 - 신=>월}{양방향 개수}$한다음 

    +- 0.1 내외면 양방향이고 양수면 월=> 신 방향이 많은거고 음수면 신=> 월 방향이 많은거고  0.2는 약간의 추세 0.5는 추세 0.8은 압도적 추세 이런식으로 만들면 될 것 같음.

    * top 10,20,50개 등 다양한 기준을 가지고 흐름을 만들어 보는 것도 좋을듯 = 아니면 빈도수를 30개 50개 이상 등으로 설정하면 다양한 흐름이 나오지 않을까 생각이 든다.

    * 계산은 대여소 번호 낮은것 - 높은 것으로 원칙을 정하고 한 곳에만 값을 매겨놓으면 되지 않을까? 아니면 아예 st_id1 -> st_id2 항목을 새로 만들어서 흐름만 관리해도 좋고

    * 기상 흐름도가 도움이 될까? 잘 맞는게 뭐가 있는지 한 번 봐보자. 아니면 화살표로 해봐도 될거같다. 대여소 주위에서 그 지역까지 화살표로 나가는걸로 하고 흐름이 쎈것만 opacity를 높게 주면 되지 않을까?

    * 화살표로 한다면 일단 지역 하나를 클릭하면 그 지역을 중심으로 어떻게 흐름이 발생하는지 보면 재밌을듯 만약 퍼지는 대여소라면 나가는 방향이 들어가는 방향보다 훨씬 많겠지 

    *  나가는 방향 빨간색 들어오는 방향 파란색으로 하면 뭔가 재밌을것 같다.


<br>


1. 둘이 합치고 꾸미면 뭔가 흐름을 볼 수 있지 않을까?

<br>


4. 따릉이로 심리적인 생활반경 그리기
예시) 예전에 주 활동 무대는 염창역 인근이었다면 지금 주 활동 무대는 신목동역이다. 

    사실 염창역도 그렇게 멀지 않지만 산이라는 자연 장애물 때문에 심리적으로 멀게 느껴진다. 

      다른 지역도 분명 그럴진데 따릉이 대여소가 어디를 가장 많이 가는지(top 20,30,50 등)를 분석하면 알 것 같다.

    * 개별 반경 100m < x <5km 따릉이 대여소를 개별 대여소 별로 찾는다. st_id_1과 st_id=2가 같으니 이것도 따로 표를 새로 만들면 될 것 같다.
    
    * 따릉이 이용소 top 20,50 등을 구하고 색을 다르게 하면 표현 될 것 같다.

    * density plot을 그릭 서울 전역을 확대하면 어느정도 생활권이 나오려나?

    * 위 방법 보다는 본인이 가장 애용하는 따릉이 대여소를 클릭하면 관련 생활권이 나오게 하는거 어떨까?

    * 많은 사람들에게도 물어봐보자 이스밍한테도 물어봐보자.



In [77]:
# #불러오기
# seoul_bike_1_2 = pd.read_hdf('D:\git_local_repository\data_for_project\seoul_bike\preprocessed_bike_2021.hdf5')
# seoul_bike_2_2 = pd.read_hdf('D:\git_local_repository\data_for_project\seoul_bike\preprocessed_bike_2_2.hdf5')

# seoul_bike = pd.concat([seoul_bike_1_2,seoul_bike_2_2])

# # 1분당 이동 거리(m)
# seoul_bike['m_pm'] = (seoul_bike['dist'] / (seoul_bike['riding_time'])).round(decimals=0)  # 1분당 거리 계산

# #한국에는 자전거에 미친놈들이 많은건가 아니면 오류인가
# # 선수급도 50km/h 속도가 나온다. 1분에 830미터 정도 달린다고 한다.
# # 동호회 수준에서도 30km/h면 수준급 실력이라 한다. 1분에 500m 정도 된다.
# # 4745948, 도심 한가운데서 분당 1000m의 속도로 54키로를 달리는게 말이되나.. 54km면 여의도 공원 => 잠원 한강공원을 왕복하고도 다시 출발해 절반은 더 가야된다.
# # 이게 말이되나.. 

# # 결론
# # 1분당 거리 계산 (1분당 400미터 이상인 경우 error로 간주)

# seoul_bike = seoul_bike.query('m_pm <= 400')

# seoul_bike


In [4]:
seoul_bike = pd.read_hdf('D:\git_local_repository\data_for_project\seoul_bike\seoul_bike_2021.hdf5')
seoul_bike = seoul_bike.query('st_id1 != 9999')
seoul_bike = seoul_bike.query('st_id2 != 9999')

### 대여소 별 이동 지역 표시 & 1km 내 따릉이 대여소 표지

In [82]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    m = 6367 * c * 1000
    return m


### 대여소 별 거리 측정
asd = station.query('id ==[104,106]')
haversine_np(asd.iloc[0,-3],asd.iloc[0,-2],asd.iloc[1,-3],asd.iloc[1,-2])
# station[station['주소2'].str.contains("월촌중")]
# station.query('id == 744')

274.03633924051456

In [127]:
def id_map(id,n, dist=1000, min_count = 500) :
    total_num_left = seoul_bike.query('st_id1==@id')['st_id2'].value_counts() # 현재 대여소에서 빌리면 주로 향하는 목적지는?
    total_num_left = total_num_left[~total_num_left.index.isin([id])] # 같은 곳에서 빌려서 같은 곳에 반납하는 경우 제거
    total_num_left = total_num_left[: np.where(total_num_left > n)[0][-1]] # 최소 빌린 횟수 

    # 2 to 1 다른 대여소 => 여의나루 
    total_num_right = seoul_bike.query('st_id2==@id')['st_id1'].value_counts() # 어느 대여소에서 현재 대여소로 이동할까?
    total_num_right = total_num_right[total_num_right.index != id] # 같은 곳에서 빌려서 같은 곳에 반납하는 경우 제거
    total_num_right = total_num_right[: np.where(total_num_right > n)[0][-1]] # 최소 빌린 횟수

    # concat
    result_concat = pd.concat([total_num_left,total_num_right],axis=1)
    result_concat.fillna((n/2),inplace=True)
    result_concat.rename(columns=dict(st_id2='1to2',st_id1='2to1'),inplace=True) 
    result_concat['rate'] = round((result_concat["1to2"] / (result_concat["1to2"]+result_concat["2to1"])),4)
    result_concat['diff'] = abs((result_concat["1to2"] - result_concat["2to1"])) # 얼마나 차이나는지 
    result_concat['total'] = abs((result_concat["1to2"] + result_concat["2to1"])) # 얼마나 차이나는지 
    result_concat['category'] = pd.cut(result_concat['rate'], bins=[0,0.45,0.55,result_concat['rate'].max()],
                                                              labels=["해당 대여소로 향하는 지역", "이동량이 비슷한 지역" ,"해당 대여소에서 주로 가는 지역"])
    add_info_result = pd.merge(result_concat,station, how='left',left_on=result_concat.index,right_on='id') # station 정보 추가

    ## 100미터 미만은 제거
    add_info_result['dist'] = haversine_np(station.query('id==@id')['경도'].values,station.query('id==@id')['위도'].values,add_info_result['경도'].values,add_info_result['위도'].values) # 거리측정
    add_info_result = add_info_result.query('dist > 100')

    # 해당 대여소 1km 반경 내 따릉이 대여소 표시
    station_haversine = haversine_np(station.query('id==@id')['경도'].values,station.query('id==@id')['위도'].values,station['경도'].values,station['위도'].values)
    station_haversine = pd.concat([station,pd.Series(station_haversine)],axis=1)
    station_haversine.rename(columns={0:'dist'},inplace=True)
    station_haversine = station_haversine.query('@dist >= dist > 0')



    ### 그래프 그리기 
    # 반경 내 따릉이 대여소 표현
    fig_2 = go.Figure(go.Scattermapbox(
        lat=station_haversine['위도'],
        lon=station_haversine['경도'],
        mode='markers',
        # marker = {'size': 20, 'symbol': "bus"},
        marker=go.scattermapbox.Marker(size=8,color='darkgreen',opacity=0.4),
        text=station_haversine['주소2'],
        ))                          

    # 지역 순위 표시
    fig=px.scatter_mapbox(add_info_result.query(' total > @min_count'), #category != "neutral" & 
                            lat="위도",lon="경도",
                            hover_name="주소2",
                            hover_data=['1to2','2to1','id'],
                            color='category',opacity=0.6,
                            size='diff',size_max=20,
                            color_continuous_scale='ylorrd',
                            zoom=12,height=500,)

    fig.add_trace(fig_2.data[0])


    # 해당 따릉이 대여소 색 표시
    fig_3 = go.Figure(go.Scattermapbox(
            lat=station.query('id==@id')['위도'],
            lon=station.query('id==@id')['경도'],
            mode='markers',
            # marker = {'size': 20, 'symbol': "bus"},
            marker=go.scattermapbox.Marker(size=10,color='purple'),
            text=[station.query('id==@id')['주소2']],
        ))                            
    fig.add_trace(fig_3.data[0])
    fig.update_layout(mapbox_style="carto-positron")
    return fig

id= 744
n = 50
# print(station[station['주소2'].str.contains('목동')][['주소2','id']])
id_map(id, n, min_count=500)

### 2. 대여소간 흐름 분석 먼저 해보자 나가고 들어오는 그런 흐름을 한 번 만들어보고 싶다.

In [5]:
# id : 대여소 id   n : 대여소간 최소 대여수 ex) 여의나루 => 1000여개 대여소 중 n회 이상 이동한 대여소만 포함
def con_dis(id,n) :
    try : 
        # 1 to 2 여의나루 => 다른 대여소
        total_num_left = seoul_bike.query('st_id1==@id')['st_id2'].value_counts() # 현재 대여소에서 빌리면 주로 향하는 목적지는?
        total_num_left = total_num_left[~total_num_left.index.isin([id])] # 같은 곳에서 빌려서 같은 곳에 반납하는 경우 제거

        # 2 to 1 다른 대여소 => 여의나루 
        total_num_right = seoul_bike.query('st_id2==@id')['st_id1'].value_counts() # 어느 대여소에서 현재 대여소로 이동할까?
        total_num_right = total_num_right[total_num_right.index != id] # 같은 곳에서 빌려서 같은 곳에 반납하는 경우 제거

        a = total_num_left.sum()
        b = total_num_right.sum()

        return [id,a,b,round(foo(a,b),3)]

    except :
        return [id]

def foo(x,y):
    try:
        return x/y
    except ZeroDivisionError:
        return 0



In [6]:
id1 = seoul_bike['st_id2'].value_counts()
id2 = seoul_bike['st_id1'].value_counts()
id1 = id1[id1>500].index # 대여횟수 최소 500회 이상인 대여소 선정
id2 = id2[id2>500].index
total_id=np.unique(np.append(id1,id2))

n = 50 # 대여소간 이동횟수 최소 n회
result_sum=[]
for num,ids in notebook.tqdm(enumerate(total_id))  :
    result_con_dis = con_dis(ids,n) # con_dis 함수 
    result_sum.append(result_con_dis)



0it [00:00, ?it/s]

In [78]:
asd = pd.merge(pd.DataFrame(result_sum),station,how='left',left_on=0,right_on='id').sort_values(by=2,ascending=False)
asd.columns = ['id_du','1to2','2to1','portion', 'id', '주소2', '주소1', '지역', '위도', '경도', '설치일']
asd.drop(columns=['id_du','주소1','설치일','portion'],inplace=True)
asd.dropna(inplace=True)
asd['rate'] = asd['1to2'] / (asd['1to2'] + asd['2to1'])
asd['category'] = pd.cut(asd['rate'], bins=[0,0.45,0.55,asd['rate'].max()],labels=["몰리는 지역", "neutral" ,"퍼지는 지역"])
asd['diff'] = abs((asd["1to2"] - asd["2to1"])) # 얼마나 차이나는지 
asd.to_csv('con_dis.csv',encoding='CP949')

In [77]:
# asd.query('0.55 > rate > 0.45').sort_values(by='rate',ascending=False)

,1to2,2to1,id,주소2,지역,위도,경도,category,diff,rate
708,1621.0,1328.0,974.0,연신초등학교옆 마을마당,은평구,37.626720,126.929016,neutral,293.0,0.549678
637,2748.0,2253.0,858.0,녹사평역3번 출구,용산구,37.534409,126.986893,neutral,495.0,0.549490
311,6155.0,5049.0,450.0,효자동 삼거리,종로구,37.583603,126.972549,neutral,1106.0,0.549357
2352,944.0,775.0,4519.0,양원보도육교,양천구,37.539246,126.823738,neutral,169.0,0.549156
913,7472.0,6135.0,1230.0,송파중학교 정문,송파구,37.495670,127.130592,neutral,1337.0,0.549129
...,...,...,...,...,...,...,...,...,...,...
542,6426.0,7769.0,745.0,강서초등학교,양천구,37.522282,126.839699,neutral,1343.0,0.452695
1872,2765.0,3352.0,2812.0,항동프라자,구로구,37.479832,126.822754,neutral,587.0,0.452019
2218,4920.0,5968.0,4023.0,월계역 1번 출구,노원구,37.634411,127.058868,neutral,1048.0,0.451874
718,1193.0,1454.0,991.0,흥국사 정류장,은평구,37.656769,126.942009,neutral,261.0,0.450699


In [80]:
fig=px.scatter_mapbox(asd.query('category != "neutral" & diff > 3000'), lat="위도", lon="경도", hover_name="주소2", hover_data=['1to2','2to1','id'], color='category', size='diff',size_max=20, color_continuous_scale='ylorrd', zoom=12, height=500, opacity=0.6)

# 해당 따릉이 대여소 색 표시하기
# fig_2 = go.Figure(go.Scattermapbox(
#         lat=station.query('id==@id')['위도'],
#         lon=station.query('id==@id')['경도'],
#         mode='markers',
#         # marker = {'size': 20, 'symbol': "bus"},
#         marker=go.scattermapbox.Marker(size=10,color='purple'),
#         text=[station.query('id==@id')['주소2']],
#     ))                            
# fig.add_trace(fig_2.data[0])
fig.update_layout(mapbox_style="carto-positron")                                                              